## Notebook 1: A first Earth Engine workflow (NDVI over Missouri)

This notebook demonstrates a **minimal but complete workflow** using Google Earth Engine:

- connect to Earth Engine
- load a public satellite dataset
- compute a simple derived product (NDVI)
- visualize the result on a map

The goal is not to be clever or exhaustive. The goal is to see the full pattern once, end to end.

In [6]:
import ee
from google.cloud import storage

# 1. Set Workspace Identifiers
PROJECT = "eeps-geospatial"
BUCKET = "wustl-eeps-edc"

# 2. Initialize Earth Engine (Required for GEE data access)
ee.Initialize(project=PROJECT)

# 3. Setup Cloud Storage Client (Required to read/write files to our bucket)
storage_client = storage.Client(project=PROJECT)

print(f"Project '{PROJECT}' initialized and storage client ready.")

Project 'eeps-geospatial' initialized and storage client ready.


## A note on Python packages

This notebook uses the `geemap` package to display interactive maps.

`geemap` is **not installed by default** in many cloud notebook environments. Because the terminal is disabled, we install it directly from the notebook using `%pip`.

This is normal in cloud workflows and does not mean anything is “broken.”

## Installing geemap (one-time step)

The next cell installs `geemap` into the current notebook environment.

- If this is your **first time running this notebook**, run the cell and then **restart the kernel**.
- After restarting, re-run the notebook from the top.
- If `geemap` is already installed, this step will be fast or skipped.

If there are packages you need to install, you can do it this way

In [3]:
%pip install -q geemap

Note: you may need to restart the kernel to use updated packages.


In [4]:
import ee
import geemap

ee.Initialize()

EEException: ee.Initialize: no project found. Call with project= or see http://goo.gle/ee-auth.

## If something goes wrong here

If you see an error like:  
``ModuleNotFoundError: No module named ‘geemap’``  
then the kernel has not been restarted yet. Restart the kernel and run the notebook again from the top.

If errors persist, stop and ask for help — do not try random fixes.

In [ ]:
# Missouri boundary (public TIGER dataset)
states = ee.FeatureCollection("TIGER/2018/States")
mo = states.filter(ee.Filter.eq("NAME", "Missouri")).geometry()

In [ ]:
# Sentinel-2 SR harmonized
s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

# Simple cloud mask using SCL (Scene Classification Layer)
def mask_s2_scl(img):
    scl = img.select("SCL")
    # Keep: vegetation, bare, water, etc. Mask: cloud/shadow/snow
    mask = scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10)).And(scl.neq(11))
    return img.updateMask(mask)

start = "2025-06-01"
end   = "2025-09-01"

s2_mo = (
    s2.filterBounds(mo)
      .filterDate(start, end)
      .map(mask_s2_scl)
)

# NDVI per image, then composite (median)
def add_ndvi(img):
    ndvi = img.normalizedDifference(["B8", "B4"]).rename("NDVI")
    return img.addBands(ndvi)

ndvi_comp = s2_mo.map(add_ndvi).select("NDVI").median().clip(mo)

In [1]:
# Map
m = geemap.Map(center=[38.5, -92.5], zoom=6)

vis = {"min": 0.0, "max": 0.8}  # keep simple for novices
m.addLayer(ndvi_comp, vis, f"NDVI median {start} to {end}")
m.addLayer(mo, {}, "Missouri boundary")
m

NameError: name 'geemap' is not defined